# Setting Up Work Environment

In [1]:
!pip install --upgrade google-generativeai

In [2]:
!pip install -q -U google-genai

In [3]:
!pip install jupyter_bokeh

In [4]:
import os
from google import genai
import google.generativeai as ggenai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

from PIL import Image
from google.genai import types

from IPython.display import HTML
import panel as pn  # GUI

In [5]:
# Set up the API key (Replace 'YOUR_API_KEY' with your actual Gemini API key)
key = userdata.get('genai_api')
client = genai.Client(api_key=key)

List the set of available models

In [6]:
ggenai.configure(api_key=key)

models = ggenai.list_models()
for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-

Create a helper function to make it easier to use prompts and look at generated outputs.

Gemini's API (via the google.generativeai Python SDK) supports only two roles in the Content object when using the generate_content() method:

✅ Supported Roles
- "user" – represents the user input or question.

- "model" – represents the model's previous responses (optional, for multi-turn context).

❌ Unsupported Roles
- "system" – not supported (unlike OpenAI's models).

- Any other custom roles (e.g., "tool", "function") – also not supported.

In [7]:
def get_completion(prompt, model="gemini-1.5-flash", temperature = 0):
  response = client.models.generate_content(
      model=model,
      contents=prompt,
      config=types.GenerateContentConfig(max_output_tokens=200, temperature = temperature)
      )
  return response.text  # Extract the generated text

In [8]:
def get_completion_from_messages(messages, model="gemini-1.5-flash", temperature = 0):
  user_prompt = "\n".join([m["content"] for m in messages])
  response = client.models.generate_content(
      model=model,
      contents=user_prompt,
      config=types.GenerateContentConfig(max_output_tokens=200, temperature = temperature)
      )
  return response.text  # Extract the generated text

📝 Simulating a System Message
Since "system" messages are unsupported, you can prepend your system instruction to the first "user" message if you need to guide the model's behavior:

system_instruction = `"You are a helpful assistant."`
user_prompt = `"What is the weather like in Cairo?"`

- Simulated prompt
full_prompt = `f"{system_instruction}\n\n{user_prompt}"`

In [9]:
def get_completion_from_messages_gem(messages, model="gemini-1.5-flash", temperature = 0):
  # Convert list of dictionaries to list of types.Content objects
  contents = []
  for message in messages:
    contents.append(types.Content(role=message['role'], parts=[types.Part(text=message['content'])]))

  response = client.models.generate_content(
      model=model,
      contents=contents, # Pass the list of Content objects
      config=types.GenerateContentConfig(max_output_tokens=200, temperature = temperature)
      )
  return response.text  # Extract the generated text

In [10]:
messages =  [
{'role':'model', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Gaber'}  ]

In [11]:
response = get_completion_from_messages_gem(messages, temperature=1)
response

"Hi Gaber, it's nice to meet you! How can I help you today?\n"

# Message Roles: Understanding Their Purpose

There are two main types of messages commonly used when interacting with a large language model. The first is the system message, which provides high-level instructions to the model and helps set its behavior and persona. Following this, the conversation proceeds with alternating turns between the user and the assistant.

If you've used the ChatGPT web interface, you're already familiar with this structure: your inputs are considered user messages, and the model’s replies are assistant messages.

The system message operates behind the scenes it acts like a quiet instruction whispered into the assistant’s ear, shaping how it responds without the user ever seeing it.

The advantage of using a system message is that it allows you, as the developer, to shape the conversation behind the scenes. You can guide the assistant’s behavior like whispering instructions in its ear—without exposing those instructions to the user. This makes it possible to influence responses subtly and consistently, without making the prompt part of the visible conversation.

In [12]:
messages = [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}
]

In [13]:
response = get_completion_from_messages(messages, temperature=1)
response

"Hark, what light through yonder poultry breaks?  'Tis not the dawn, but a feathered conundrum! Why, pray tell, did yon chicken traverse the thoroughfare?  I know not, good sir, but perchance 'twas to seek a more succulent grain, or to escape the clutches of a villainous fox, or – nay, 'tis a jest! Perhaps 'twas simply to prove its inherent fowl-hearted bravery!  The answer, alas, remains shrouded in the mists of poultry-based mystery!\n"

In [14]:
messages = [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Gaber'}
]

In [15]:
response = get_completion_from_messages(messages, temperature=1)
response

"Hi Gaber! It's nice to meet you. How can I help you today?\n"

In [16]:
messages = [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}
]

In [17]:
response = get_completion_from_messages(messages, temperature=1)
response

"As a chatbot, I have no access to your personal information, including your name.  To help me remember things, I need you to tell me!  What's your name? 😊\n"

In this example, you'll notice that the model doesn't know your name by default. That's because each interaction with a language model is **stateless** meaning every conversation is treated as a standalone session.

To enable the model to "remember" earlier parts of a conversation, you must explicitly include all relevant prior messages as part of the input. This collection of previous messages is referred to as the context.

Let’s try this: first, we’ll provide the necessary context by stating your name in an earlier message. Then, we’ll ask the model, "What’s my name?" Since the context is now included, the model will be able to generate a correct response using that prior information.

In [18]:
messages = [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Gaber'},
{'role':'assistant', 'content': "Hi Gaber! It's nice to meet you. Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}
]

In [19]:
response = get_completion_from_messages(messages, temperature=1)
response

'Hi Gaber!  Your name is Gaber.  Anything else I can help you with?\n'

# Build a Chatbot with Custom Features

Now, let’s build our chatbot **Pizzabot** which will take orders for a pizza restaurant. The goal is to automate the process of collecting user inputs and generating assistant responses.

To begin, we’ll define a helper function called `collect_messages`. This function will:

- Receive user input (prompts) through a user interface we'll build shortly.

- Append each user message to a list called `context`.

- Call the language model using this updated context.

Each time the model responds, its message will also be appended to the  `context`. In other words, both user and assistant messages will be added sequentially to the context list, which grows over time. This accumulating history provides the model with all necessary background to respond appropriately and continue the conversation intelligently.

By maintaining this dynamic context, Pizzabot can understand the flow of the conversation and handle multi-turn interactions effectively.

In [31]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    # Wrap the Markdown pane in a Column and apply the style to the Column
    panels.append(
        pn.Row('Assistant:', pn.Column(pn.pane.Markdown(response, width=600), css_classes=['assistant-response-bg'])))

    return pn.Column(*panels)

Now, we’ll set up and run the user interface to display Pizzabot. This chatbot will use a `context` list that includes a **system message** which acts as the instruction manual for the assistant—and this message contains the full menu.

Every time we call the language model, we’ll use the same `context`, which will grow over time as new user and assistant messages are added. This allows the model to maintain continuity and understand the full conversation history.

The system message for Pizzabot is designed to guide the assistant in handling pizza orders. It instructs the assistant to:

- Greet the customer

- Collect the order details

- Ask whether it’s for pickup or delivery

- Wait until the full order is provided before summarizing

After summarizing, the assistant should check if the customer wants to add anything else. If it's a delivery, the assistant should ask for an address. Finally, the assistant should collect payment information and clarify all item details such as size, extras, and options to ensure that each item is clearly identified from the menu.

The assistant is also instructed to respond in a short, friendly, and conversational tone.

Once everything is set up and the UI is executed, you’ll see the chatbot GUI appear below.

In [32]:
pn.extension(css_classes=['assistant-response-bg'], raw_css=[
    '.assistant-response-bg { background-color: #F6F6F6; }'
])

In [33]:
panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

In [34]:
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

Finally, we can instruct the model to generate a **JSON summary** of the order, which can then be sent to the order processing system. To do this, we append an additional system message that provides explicit instructions to the model.

This system message should ask the assistant to create a JSON object summarizing the food order based on the conversation history. The summary should include the following fields:

- `pizza`: the type(s) of pizza ordered

- `toppings`: a list of selected toppings

- `drinks`: a list of any drinks included

- `sides`: a list of side items

- `total_price`: the final calculated total

Since we want the output to be structured and consistent, we use a lower temperature setting to reduce randomness and ensure the response is predictable an important aspect for tasks involving structured data generation within a conversational agent.

In [35]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)

In [38]:
response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
  "pizza": [
    {
      "type": "pepperoni",
      "size": "small",
      "price": 7.00
    }
  ],
  "toppings": [],
  "drinks": [],
  "sides": [],
  "totalPrice": 7.00
}
```

